# ⚖️ Lab 14: RAG vs Fine-Tuning
## Module 14 - Decision Framework for Banking Use Cases

**Duration:** 15 minutes

**Objectives:**
- Compare RAG and fine-tuning approaches
- Evaluate for banking scenarios
- Implement hybrid approach

**Banking Scenario:** Document summarization comparison

---

In [ ]:
import os
import json
import time

# =============================================================================
# GOOGLE COLAB SETUP - Add these secrets (click 🔑 icon):
#   - AZURE_OPENAI_KEY: Your API key
#   - AZURE_OPENAI_ENDPOINT: https://xxx.openai.azure.com/
#   - AZURE_OPENAI_DEPLOYMENT: Your model deployment name
# =============================================================================

DEMO_MODE = False
client = None
MODEL_NAME = "gpt-4o"

try:
    from google.colab import userdata
    AZURE_OPENAI_KEY = userdata.get('AZURE_OPENAI_KEY')
    AZURE_OPENAI_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')
    try:
        MODEL_NAME = userdata.get('AZURE_OPENAI_DEPLOYMENT')
    except:
        pass
    if AZURE_OPENAI_KEY and AZURE_OPENAI_ENDPOINT:
        if not AZURE_OPENAI_ENDPOINT.startswith('http'):
            AZURE_OPENAI_ENDPOINT = 'https://' + AZURE_OPENAI_ENDPOINT
        print(f"✅ Credentials loaded. Model: {MODEL_NAME}")
    else:
        raise ValueError("Missing")
except Exception:
    print("⚠️ Running in DEMO MODE")
    DEMO_MODE = True

if not DEMO_MODE:
    from openai import AzureOpenAI
    client = AzureOpenAI(
        api_key=AZURE_OPENAI_KEY,
        api_version="2024-06-01",
        azure_endpoint=AZURE_OPENAI_ENDPOINT
    )
    print("✅ Client ready")

## Part 1: RAG Approach

In [ ]:
# Simulated vector store with example summaries
example_summaries = [
    {"input": "Mortgage $300K 30yr 6.5%", "output": "**Type:** Mortgage\n**Amount:** $300,000\n**Term:** 30 years\n**Rate:** 6.5%"},
    {"input": "Auto loan $25K 5yr 7.9%", "output": "**Type:** Auto\n**Amount:** $25,000\n**Term:** 5 years\n**Rate:** 7.9%"}
]

# Use available model (fallback to gpt-4o if gpt-4o-mini not available)
AVAILABLE_MODEL = MODEL_NAME if MODEL_NAME else "gpt-4o"

def rag_summarize(document: str) -> dict:
    """RAG approach: Retrieve examples, then generate"""
    start = time.time()
    
    # Simulate retrieval
    examples_text = "\n".join([f"Input: {e['input']}\nOutput: {e['output']}" for e in example_summaries])
    
    # Try gpt-4o-mini first, fallback to available model
    model_to_use = AVAILABLE_MODEL
    try:
        # Test if gpt-4o-mini is available
        test_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": "test"}],
            max_tokens=1
        )
        model_to_use = "gpt-4o-mini"
    except:
        print("⚠️ gpt-4o-mini not available, using", AVAILABLE_MODEL)
        model_to_use = AVAILABLE_MODEL
    
    response = client.chat.completions.create(
        model=model_to_use,
        messages=[{
            "role": "system",
            "content": f"Summarize loan documents like these examples:\n{examples_text}"
        }, {
            "role": "user",
            "content": f"Summarize: {document}"
        }]
    )
    
    return {
        "output": response.choices[0].message.content,
        "latency": time.time() - start,
        "tokens": response.usage.total_tokens,
        "approach": f"RAG ({model_to_use})",
        "model_used": model_to_use
    }

print(f"✅ RAG function defined (will use {AVAILABLE_MODEL} if gpt-4o-mini unavailable)")

## Part 2: Fine-Tuned Approach (Simulated)

In [ ]:
def finetuned_summarize(document: str) -> dict:
    """Fine-tuned approach: Direct generation"""
    start = time.time()
    
    # In production: use fine-tuned model
    # model = "ft:gpt-4o-mini:banking:loan-summarizer"
    
    # Try gpt-4o-mini first, fallback to available model
    model_to_use = AVAILABLE_MODEL
    try:
        # Test if gpt-4o-mini is available
        test_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": "test"}],
            max_tokens=1
        )
        model_to_use = "gpt-4o-mini"
    except:
        model_to_use = AVAILABLE_MODEL
    
    response = client.chat.completions.create(
        model=model_to_use,
        messages=[{
            "role": "system",
            "content": "Summarize loan documents in this format: **Type:** X\n**Amount:** $X\n**Term:** X years\n**Rate:** X%"
        }, {
            "role": "user",
            "content": document
        }]
    )
    
    return {
        "output": response.choices[0].message.content,
        "latency": time.time() - start,
        "tokens": response.usage.total_tokens,
        "approach": f"Fine-tuned ({model_to_use})",
        "model_used": model_to_use
    }

print(f"✅ Fine-tuned function defined (will use {AVAILABLE_MODEL} if gpt-4o-mini unavailable)")

## Part 3: Compare Approaches

In [ ]:
test_doc = "Home equity line of credit for $100,000 at 9.25% variable rate for 10 years. Applicant has $250,000 home equity."

print("Testing both approaches...\n")

try:
    rag_result = rag_summarize(test_doc)
    ft_result = finetuned_summarize(test_doc)
    
    print("="*60)
    print("COMPARISON RESULTS")
    print("="*60)
    print(f"\n{'Metric':<15} {'RAG':<25} {'Fine-tuned':<25}")
    print("-"*70)
    print(f"{'Model Used':<15} {rag_result.get('model_used', 'N/A'):<25} {ft_result.get('model_used', 'N/A'):<25}")
    print(f"{'Latency':<15} {rag_result['latency']:.2f}s{'':<19} {ft_result['latency']:.2f}s")
    print(f"{'Tokens':<15} {rag_result['tokens']:<25} {ft_result['tokens']:<25}")
    
    print("\n" + "="*60)
    print("RAG Output:")
    print(rag_result['output'])
    print("\n" + "="*60)
    print("Fine-tuned Output:")
    print(ft_result['output'])
    
except Exception as e:
    print(f"❌ Error running comparison: {e}")
    print("\n💡 Make sure you have deployed at least one model (gpt-4o or gpt-4o-mini)")
    print("\n🔧 To deploy gpt-4o-mini:")
    print("   az cognitiveservices account deployment create \\")
    print("     --name banking-openai-service \\")
    print("     --resource-group rg-banking-ai \\")
    print("     --deployment-name gpt-4o-mini \\")
    print("     --model-name gpt-4o-mini \\")
    print("     --model-version '2024-07-18' \\")
    print("     --model-format OpenAI \\")
    print("     --sku-capacity 10 \\")
    print("     --sku-name Standard")

## Part 4: Decision Framework

In [ ]:
def recommend_approach(use_case: dict) -> str:
    """Recommend RAG, Fine-tuning, or Hybrid"""
    
    knowledge_changes = use_case.get("knowledge_changes_frequently", False)
    needs_format = use_case.get("needs_consistent_format", False)
    needs_sources = use_case.get("needs_source_citations", False)
    latency_critical = use_case.get("latency_critical", False)
    
    if knowledge_changes or needs_sources:
        if needs_format:
            return "HYBRID: RAG for knowledge + Fine-tuned for format"
        return "RAG: Knowledge changes frequently or needs citations"
    
    if needs_format or latency_critical:
        return "FINE-TUNE: Consistent format or low latency needed"
    
    return "PROMPT ENGINEERING: Start simple, iterate"

# Test recommendations
use_cases = [
    {"name": "Policy FAQ Bot", "knowledge_changes_frequently": True, "needs_source_citations": True},
    {"name": "Loan Classifier", "needs_consistent_format": True, "latency_critical": True},
    {"name": "Document Processor", "knowledge_changes_frequently": True, "needs_consistent_format": True}
]

print("\n" + "="*60)
print("RECOMMENDATIONS")
print("="*60)
for uc in use_cases:
    rec = recommend_approach(uc)
    print(f"\n{uc['name']}:")
    print(f"  → {rec}")

---
## ✅ Lab 14 Complete!

**Key Takeaways:**
- RAG: Best for changing knowledge, source citations
- Fine-tuning: Best for consistent format, low latency
- Hybrid: Combine both for complex use cases
- Always start with prompt engineering first